# NOTE:  THIS NOTEBOOK WILL TAKE A 30 MINUTES TO COMPLETE.

# PLEASE BE PATIENT.

# Fine-Tuning a BERT Model and Create a Text Classifier

In the previous section, we've already performed the Feature Engineering to create BERT embeddings from the `reviews_body` text using the pre-trained BERT model, and split the dataset into train, validation and test files. To optimize for Tensorflow training, we saved the files in TFRecord format. 

Now, let’s fine-tune the BERT model to our Customer Reviews Dataset and add a new classification layer to predict the `star_rating` for a given `review_body`.

![BERT Training](img/bert_training.png)

As mentioned earlier, BERT’s attention mechanism is called a Transformer. This is, not coincidentally, the name of the popular BERT Python library, “Transformers,” maintained by a company called HuggingFace. We will use a variant of BERT called [DistilBert](https://arxiv.org/pdf/1910.01108.pdf) which requires less memory and compute, but maintains very good accuracy on our dataset.

In [1]:
!pip install -q sagemaker==2.20.0
!pip install -q smdebug==1.0.1

In [2]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# _PRE-REQUISITE: You need to have succesfully run the notebooks in the `PREPARE` section before you continue with this notebook._

In [3]:
%store -r processed_train_data_s3_uri

In [4]:
try:
    processed_train_data_s3_uri
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the PREPARE section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [5]:
print(processed_train_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-train


In [6]:
%store -r processed_validation_data_s3_uri

In [7]:
try:
    processed_validation_data_s3_uri
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the PREPARE section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [8]:
print(processed_validation_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-validation


In [9]:
%store -r processed_test_data_s3_uri

In [10]:
try:
    processed_test_data_s3_uri
except NameError:
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the PREPARE section before you continue.')
    print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

In [11]:
print(processed_test_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-test


# Track the `Experiment`
We will track every step of this experiment throughout the `prepare`, `train`, `optimize`, and `deploy`.

# Concepts

**Experiment**: A collection of related Trials.  Add Trials to an Experiment that you wish to compare together.

**Trial**: A description of a multi-step machine learning workflow. Each step in the workflow is described by a Trial Component. There is no relationship between Trial Components such as ordering.

**Trial Component**: A description of a single step in a machine learning workflow. For example data cleaning, feature extraction, model training, model evaluation, etc.

**Tracker**: A logger of information about a single TrialComponent.

<img src="img/sagemaker-experiments.png" width="90%" align="left">


# Create the `Experiment`

In [12]:
# import time
# from smexperiments.experiment import Experiment

# timestamp = int(time.time())

# experiment = Experiment.create(
#                 experiment_name='Amazon-Customer-Reviews-BERT-Experiment-{}'.format(timestamp),
#                 description='Amazon Customer Reviews BERT Experiment', 
#                 sagemaker_boto_client=sm)

# experiment_name = experiment.experiment_name
# print('Experiment name: {}'.format(experiment_name))

Experiment name: Amazon-Customer-Reviews-BERT-Experiment-1609549145


# Create the `Trial`

In [13]:
# import time
# from smexperiments.trial import Trial

# timestamp = int(time.time())

# trial = Trial.create(trial_name='trial-{}'.format(timestamp),
#                      experiment_name=experiment_name,
#                      sagemaker_boto_client=sm)

# trial_name = trial.trial_name
# print('Trial name: {}'.format(trial_name))

Trial name: trial-1609549145


# Create the `prepare` Trial Component and Tracker
Note:  A Trial Component is actually created through a Tracker.  This is a bit confusing, we know.

In [14]:
# from smexperiments.tracker import Tracker

# tracker_prepare = Tracker.create(display_name='prepare', 
#                                  sagemaker_boto_client=sm)

# prepare_trial_component_name = tracker_prepare.trial_component.trial_component_name
# print('Prepare trial component name {}'.format(prepare_trial_component_name))

Prepare trial component name TrialComponent-2021-01-02-005905-zwpm


# Attach the `prepare` Trial Component and Tracker as a Component to the Trial

In [15]:
# trial.add_trial_component(tracker_prepare.trial_component)

# Log All Parameters Used During `prepare` Phase

In [16]:
# %store -r raw_input_data_s3_uri

In [17]:
# print(raw_input_data_s3_uri)

s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/


In [18]:
# tracker_prepare.log_input(name='raw_data_s3_uri', 
#                           media_type='s3/uri', 
#                           value=raw_input_data_s3_uri)

# # must save after logging
# tracker_prepare.trial_component.save()

TrialComponent(sagemaker_boto_client=<botocore.client.SageMaker object at 0x7f73efcb3b00>,trial_component_name='TrialComponent-2021-01-02-005905-zwpm',display_name='prepare',tags=None,trial_component_arn='arn:aws:sagemaker:us-east-1:835319576252:experiment-trial-component/trialcomponent-2021-01-02-005905-zwpm',response_metadata={'RequestId': '9adde967-b6b1-4492-b1ce-205ebe1af314', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9adde967-b6b1-4492-b1ce-205ebe1af314', 'content-type': 'application/x-amz-json-1.1', 'content-length': '129', 'date': 'Sat, 02 Jan 2021 00:59:06 GMT'}, 'RetryAttempts': 0},parameters={},input_artifacts={'raw_data_s3_uri': TrialComponentArtifact(value='s3://sagemaker-us-east-1-835319576252/amazon-reviews-pds/tsv/',media_type='s3/uri')},output_artifacts={})

In [19]:
# %store -r train_split_percentage

no stored variable or alias train_split_percentage


In [20]:
# print(train_split_percentage)

NameError: name 'train_split_percentage' is not defined

In [ ]:
# %store -r validation_split_percentage

In [ ]:
# print(validation_split_percentage)

In [ ]:
# %store -r test_split_percentage

In [ ]:
# print(test_split_percentage)

In [26]:
%store -r max_seq_length

In [27]:
print(max_seq_length)

64


In [ ]:
# %store -r balance_dataset

In [ ]:
# print(balance_dataset)

In [ ]:
# tracker_prepare.log_parameters({
#     'max_seq_length': max_seq_length,
#     'train_split_percentage': train_split_percentage,
#     'validation_split_percentage': validation_split_percentage,
#     'test_split_percentage': test_split_percentage, 
#     'balance_dataset': str(balance_dataset)
# })

# # must save after logging
# tracker_prepare.trial_component.save()

In [ ]:
# tracker_prepare.log_output(name='train_data_s3_uri', 
#                            media_type='s3/uri', 
#                            value=processed_train_data_s3_uri)

# tracker_prepare.log_output(name='validation_data_s3_uri', 
#                            media_type='s3/uri', 
#                            value=processed_validation_data_s3_uri)

# tracker_prepare.log_output(name='test_data_s3_uri', 
#                            media_type='s3/uri', 
#                            value=processed_test_data_s3_uri)

# # must save after logging
# tracker_prepare.trial_component.save()

# Specify the Dataset in S3
We are using the train, validation, and test splits created in the previous section.

In [21]:
print(processed_train_data_s3_uri)

!aws s3 ls $processed_train_data_s3_uri/

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-train
2020-12-31 08:49:55     353032 part-algo-1-amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tfrecord
2020-12-31 08:49:55      10844 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-12-31 08:49:55      11611 part-algo-1-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [22]:
print(processed_validation_data_s3_uri)

!aws s3 ls $processed_validation_data_s3_uri/

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-validation
2020-12-31 08:49:55      19982 part-algo-1-amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tfrecord
2020-12-31 08:49:55        646 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-12-31 08:49:55        716 part-algo-1-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


In [23]:
print(processed_test_data_s3_uri)

!aws s3 ls $processed_test_data_s3_uri/

s3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-test
2020-12-31 08:49:56      20020 part-algo-1-amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tfrecord
2020-12-31 08:49:56        692 part-algo-1-amazon_reviews_us_Digital_Software_v1_00.tfrecord
2020-12-31 08:49:56        678 part-algo-1-amazon_reviews_us_Digital_Video_Games_v1_00.tfrecord


# Specify S3 `Distribution Strategy`

In [28]:
from sagemaker.inputs import TrainingInput

s3_input_train_data = TrainingInput(s3_data=processed_train_data_s3_uri, 
                                         distribution='ShardedByS3Key') 
s3_input_validation_data = TrainingInput(s3_data=processed_validation_data_s3_uri, 
                                              distribution='ShardedByS3Key')
s3_input_test_data = TrainingInput(s3_data=processed_test_data_s3_uri, 
                                        distribution='ShardedByS3Key')

print(s3_input_train_data.config)
print(s3_input_validation_data.config)
print(s3_input_test_data.config)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-train', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-validation', 'S3DataDistributionType': 'ShardedByS3Key'}}}
{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-us-east-1-835319576252/sagemaker-scikit-learn-2020-12-31-08-43-25-560/output/bert-test', 'S3DataDistributionType': 'ShardedByS3Key'}}}


# Setup Hyper-Parameters for Classification Layer

In [29]:
print(max_seq_length)

64


In [30]:
epochs=3
learning_rate=0.00001
epsilon=0.00000001
train_batch_size=128
validation_batch_size=128
test_batch_size=128
train_steps_per_epoch=100
validation_steps=100
test_steps=100
train_instance_count=1
train_instance_type='ml.p3.2xlarge'
train_volume_size=1024
use_xla=True
use_amp=True
freeze_bert_layer=False
enable_sagemaker_debugger=True
enable_checkpointing=False
enable_tensorboard=False
input_mode='File'
run_validation=True
run_test=True
run_sample_predictions=True

# Setup Metrics To Track Model Performance

These sample log lines...
```
45/50 [=====>..] - ETA: 3s - loss: 0.425 - accuracy: 0.881
50/50 [=======>] - ETA: 0s - val_loss: 0.407 - val_accuracy: 0.885
```
...will produce the following 4 metrics in CloudWatch:

`loss` = 0.425

`accuracy` = 0.881

`val_loss` = 0.407

`val_accuracy` = 0.885

<img src="img/cloudwatch_train_accuracy.png" width="50%" align="left">

<img src="img/cloudwatch_train_loss.png" width="50%" align="left">

In [31]:
metrics_definitions = [
     {'Name': 'train:loss', 'Regex': 'loss: ([0-9\\.]+)'},
     {'Name': 'train:accuracy', 'Regex': 'accuracy: ([0-9\\.]+)'},
     {'Name': 'validation:loss', 'Regex': 'val_loss: ([0-9\\.]+)'},
     {'Name': 'validation:accuracy', 'Regex': 'val_accuracy: ([0-9\\.]+)'},
]

# Setup SageMaker Debugger
Define Debugger Rules as deccribed here:  https://docs.aws.amazon.com/sagemaker/latest/dg/debugger-built-in-rules.html

In [32]:
# from sagemaker.debugger import Rule
# from sagemaker.debugger import rule_configs
# from sagemaker.debugger import CollectionConfig
from sagemaker.debugger import DebuggerHookConfig

# rules=[
#         Rule.sagemaker(
#             rule_configs.loss_not_decreasing(),
#             rule_parameters={
#                 'collection_names': 'losses,metrics',
#                 'use_losses_collection': 'true',
#                 'num_steps': '10',
#                 'diff_percent': '50'
#             },
#             collections_to_save=[
#                 CollectionConfig(name='losses',
#                                  parameters={
#                                      'save_interval': '10',
#                                  }),
#                 CollectionConfig(name='metrics',
#                                  parameters={
#                                      'save_interval': '10',
#                                  })
#             ]
#         ),
#         Rule.sagemaker(
#             rule_configs.overtraining(),
#             rule_parameters={
#                 'collection_names': 'losses,metrics',
#                 'patience_train': '10',
#                 'patience_validation': '10',
#                 'delta': '0.5'
#             },
#             collections_to_save=[
#                 CollectionConfig(name='losses',
#                                  parameters={
#                                      'save_interval': '10',
#                                  }),
#                 CollectionConfig(name='metrics',
#                                  parameters={
#                                      'save_interval': '10',
#                                  })
#             ]
#         )
#     ]

hook_config = DebuggerHookConfig(
    hook_parameters={
        'save_interval': '10', # number of steps
        'export_tensorboard': 'true',
        'tensorboard_dir': 'hook_tensorboard/',
    })

## Configure Debugger Rules
We specify the following rules:

* loss_not_decreasing: checks if loss is decreasing and triggers if the loss has not decreased by a certain persentage in the last few iterations
* LowGPUUtilization: checks if GPU is under-utilizated
* ProfilerReport: runs the entire set of performance rules and create a final output report with further insights and recommendations.

In [33]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs

rules=[ 
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overtraining()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization())
]

## Specify a Debugger profiler configuration

The following configuration will capture system metrics at 500 milliseconds. The system metrics include utilization per CPU, GPU, memory utilization per CPU, GPU as well I/O and network.

Debugger will capture detailed profiling information from step 5 to step 15. This information includes Horovod metrics, dataloading, preprocessing, operators running on CPU and GPU.

In [34]:
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500,
    framework_profile_params=FrameworkProfile(local_path="/opt/ml/output/profiler/", start_step=5, num_steps=10)
)

# Specify Checkpoint S3 Location
This is used for Spot Instances Training.  If nodes are replaced, the new node will start training from the latest checkpoint.

In [35]:
import uuid

checkpoint_s3_prefix = 'checkpoints/{}'.format(str(uuid.uuid4()))
checkpoint_s3_uri = 's3://{}/{}/'.format(bucket, checkpoint_s3_prefix)

print(checkpoint_s3_uri)

s3://sagemaker-us-east-1-835319576252/checkpoints/6be259c8-0b8a-4f75-998d-b8c51038898c/


# Setup Our BERT + TensorFlow Script to Run on SageMaker
Prepare our TensorFlow model to run on the managed SageMaker service

In [36]:
!pygmentize src-tf231-profiler/tf_bert_reviews.py

import time
import random
import pandas as pd
from glob import glob
import pprint
import argparse
import json
import subprocess
import sys
import os
import tensorflow as tf
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.3.0.1.6.1'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker-tensorflow==2.1.0.1.0.0'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==0.9.3'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'smdebug==1.0.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.23.1'])
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'matplotlib==3.2.1'])
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from

                config = DistilBertConfig.from_pretrained('distilbert-base-uncased',
                                                          num_labels=len(CLASSES))
                model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',
                                                                              config=config)
                successful_download = True
                print('Sucessfully downloaded after {} retries.'.format(retries))
            except:
                retries = retries + 1
                random_sleep = random.randint(1, 30)
                print('Retry #{}.  Sleeping for {} seconds'.format(retries, random_sleep))
                time.sleep(random_sleep)

        callbacks = []

        initial_epoch_number = 0 

        if enable_checkpointing:
            print('***** Checkpoint enabled *****')
            
            os.makedirs(checkpoint_path, exist_ok=True)        
            if os.listdir(checkpoint_path):


In [37]:
tf_image = sagemaker.image_uris.retrieve('tensorflow', region=region, version='2.3.1', py_version='py37', image_scope='training', instance_type='ml.p3.2xlarge')
print(tf_image)

763104351884.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.3.1-gpu-py37


In [38]:
print(role)

arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881


In [39]:
from sagemaker.tensorflow import TensorFlow

estimator = TensorFlow(entry_point='tf_bert_reviews.py',
                       source_dir='src-tf231-profiler',
                       role=role,
                       image_uri=tf_image,
                       instance_count=train_instance_count,
                       instance_type=train_instance_type,
                       volume_size=train_volume_size,
#                        use_spot_instances=True,
#                        max_wait=7200, # Seconds to wait for spot instances to become available
                       checkpoint_s3_uri=checkpoint_s3_uri,
#                       py_version='py37',
#                       framework_version='2.3.1',
                       hyperparameters={'epochs': epochs,
                                        'learning_rate': learning_rate,
                                        'epsilon': epsilon,
                                        'train_batch_size': train_batch_size,
                                        'validation_batch_size': validation_batch_size,
                                        'test_batch_size': test_batch_size,                                             
                                        'train_steps_per_epoch': train_steps_per_epoch,
                                        'validation_steps': validation_steps,
                                        'test_steps': test_steps,
                                        'use_xla': use_xla,
                                        'use_amp': use_amp,                                             
                                        'max_seq_length': max_seq_length,
                                        'freeze_bert_layer': freeze_bert_layer,
                                        'enable_sagemaker_debugger': enable_sagemaker_debugger,
                                        'enable_checkpointing': enable_checkpointing,
                                        'enable_tensorboard': enable_tensorboard,                                        
                                        'run_validation': run_validation,
                                        'run_test': run_test,
                                        'run_sample_predictions': run_sample_predictions},
                       input_mode=input_mode,
                       metric_definitions=metrics_definitions,
                       rules=rules,
                       debugger_hook_config=hook_config, 
                       profiler_config=profiler_config
#                       max_run=7200, # number of seconds
                      )

# Create the `Experiment Config`

In [40]:
experiment_config = {
    'ExperimentName': experiment_name,
    'TrialName': trial.trial_name,
    'TrialComponentDisplayName': 'train'
}

In [41]:
print(experiment_name)

Amazon-Customer-Reviews-BERT-Experiment-1609549145


In [42]:
%store experiment_name

Stored 'experiment_name' (str)


In [43]:
print(trial_name)

trial-1609549145


In [44]:
%store trial_name

Stored 'trial_name' (str)


# Train the Model on SageMaker

In [45]:
estimator.fit(inputs={'train': s3_input_train_data, 
                      'validation': s3_input_validation_data,
                      'test': s3_input_test_data
              },              
              experiment_config=experiment_config,                   
              wait=False)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: latest.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2021-01-02-01-02-31-908


In [46]:
training_job_name = estimator.latest_training_job.name
print('Training Job Name:  {}'.format(training_job_name))

Training Job Name:  tensorflow-training-2021-01-02-01-02-31-908


In [47]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/jobs/{}">Training Job</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [48]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/TrainingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, training_job_name)))


In [49]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Training Job Has Completed</b>'.format(bucket, training_job_name, region)))


In [50]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Checkpoint Data</a> After The Training Job Has Completed</b>'.format(bucket, checkpoint_s3_prefix, region)))


In [51]:
%%time

estimator.latest_training_job.wait(logs=False)


2021-01-02 01:02:36 Starting - Launching requested ML instances.................
2021-01-02 01:04:04 Starting - Preparing the instances for training.................
2021-01-02 01:05:36 Downloading - Downloading input data........
2021-01-02 01:06:22 Training - Downloading the training image.......
2021-01-02 01:07:03 Training - Training image download completed. Training in progress..................................................................................
2021-01-02 01:13:54 Uploading - Uploading generated training model...............
2021-01-02 01:15:13 Completed - Training job completed
CPU times: user 608 ms, sys: 21.7 ms, total: 630 ms
Wall time: 12min 39s


In [52]:
# sage = boto3.client('sagemaker')
# sage.describe_training_job(TrainingJobName='tensorflow-training-2020-12-18-18-01-49-949')

# Wait Until the ^^ Training Job ^^ Completes Above!

# [INFO] _Feel free to continue to the next workshop section while this notebook is running._

In [53]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz
    

download: s3://sagemaker-us-east-1-835319576252/tensorflow-training-2021-01-02-01-02-31-908/output/model.tar.gz to ./model.tar.gz


In [54]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

metrics/
metrics/confusion_matrix.png
transformers/
transformers/fine-tuned/
transformers/fine-tuned/tf_model.h5
transformers/fine-tuned/config.json
tensorflow/
tensorflow/saved_model/
tensorflow/saved_model/0/
tensorflow/saved_model/0/saved_model.pb
tensorflow/saved_model/0/variables/
tensorflow/saved_model/0/variables/variables.data-00000-of-00001
tensorflow/saved_model/0/variables/variables.index
tensorflow/saved_model/0/assets/
tensorboard/
code/
code/inference.py


In [55]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

2021-01-02 01:16:10.094839: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['input_ids'] tensor_info:
        dtype: DT_INT32
        shape: (-1, 5)
        name: serving_default_input_ids:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_1'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 5)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/servi

In [56]:
!saved_model_cli run --dir ./model/tensorflow/saved_model/0/ --tag_set serve --signature_def serving_default \
    --input_exprs 'input_ids=np.zeros((1,64))' # ;input_mask=np.zeros((1,64))' # ;segment_ids=np.zeros((1,64))'

2021-01-02 01:16:17.976719: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-01-02 01:16:19.289618: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-01-02 01:16:19.368877: E tensorflow/stream_executor/cuda/cuda_driver.cc:314] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-02 01:16:19.368919: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-16-38-126): /proc/driver/nvidia/version does not exist
2021-01-02 01:16:19.369413: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler fl

# Analyze Debugger Rules

In [ ]:
estimator.latest_training_job.rule_job_summary()

In [ ]:
training_job_debugger_artifacts_path = estimator.latest_job_debugger_artifacts_path()
print(training_job_debugger_artifacts_path)

# Analyze Debugger Profiling Data

Copy outputs of the following cell (`training_job_name` and `region`) to run the analysis notebooks `profiling_generic_dashboard.ipynb`, `analyze_performance_bottlenecks.ipynb`, and `profiling_interactive_analysis.ipynb`.

In [ ]:
training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

While the training is still in progress you can visualize the performance data in SageMaker Studio or in the notebook.
Debugger provides utilities to plot system metrics in form of timeline charts or heatmaps. Checkout out the notebook 
[profiling_interactive_analysis.ipynb](analysis_tools/profiling_interactive_analysis.ipynb) for more details. In the following code cell we plot the total CPU and GPU utilization as timeseries charts. To visualize other metrics such as I/O, memory, network you simply need to extend the list passed to `select_dimension` and `select_events`.

In [ ]:
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob
tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()

In [ ]:
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts  = TimelineCharts(system_metrics_reader, 
                                       framework_metrics_reader=None,
                                       select_dimensions=["CPU", "GPU"],
                                       select_events=["total"])

# Download Debugger Profiling Report and Notebook

The profiling report rule will create an html report `profiler-report.html` with a summary of builtin rules and recommenades of next steps. You can find this report in your S3 bucket.  

In [ ]:
profiler_report = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output/ProfilerReport/profiler-output/profiler-report.html"
print(profiler_report)

In [ ]:
!aws s3 cp $profiler_report .

In [ ]:
profiler_notebook = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output/ProfilerReport/profiler-output/profiler-report.ipynb"
print(profiler_notebook)

In [ ]:
!aws s3 cp $profiler_notebook .

# Download Trained Model

In [ ]:
!aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz ./model.tar.gz

In [ ]:
!mkdir -p ./model/
!tar -xvzf ./model.tar.gz -C ./model/

In [ ]:
!saved_model_cli show --all --dir ./model/tensorflow/saved_model/0/

# Show the Experiment Tracking Lineage

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

lineage_table = ExperimentAnalytics(
    sagemaker_session=sess,
    experiment_name=experiment_name,
    metric_names=['validation:accuracy'],
    sort_by="CreationTime",
    sort_order="Ascending",
)

lineage_df = lineage_table.dataframe()
lineage_df.shape

In [ ]:
lineage_df

In [ ]:
sm.describe_trial_component(TrialComponentName=lineage_df.TrialComponentName[0])

# Pass Variables to the Next Notebook(s)

In [ ]:
%store training_job_name

In [ ]:
%store prepare_trial_component_name

In [ ]:
%store training_job_debugger_artifacts_path

In [ ]:
%store

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();